# DESaster Simulation Set Up Template

## Required Modules

In [365]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

## Input Data

In [366]:
scenario_file = '../inputs/scenario_test_renters.xlsx'

In [367]:
# Create Pandas dataframe of attribute data for all owners to be modeled in the simulation
owners_df = pd.read_excel(scenario_file, sheetname='owners')
owners_df

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Alfred,30000,0,0.00,62 That St,Mobile Home,1500,1,1,700,1920,100000,Complete,Own,Own,Single Family Dwelling
1,Bruce,100000,1000000,0.85,720 This Rd,Single Family Dwelling,3500,5,5,5000,1920,10000000,None,Own,Own,Single Family Dwelling
2,Selena,10000,100,0.00,1001 Other Ave,Single Family Dwelling,1000,0,1,250,1960,10000,None,Own,Own,Single Family Dwelling
3,Fish,50000,1000,0.85,26000 Out There Lane,Single Family Dwelling,3000,4,2,2000,2010,800000,None,Own,Own,Single Family Dwelling


In [368]:
# Create Pandas dataframe of attribute data for all renters to be modeled in the simulation
renters_df = pd.read_excel(scenario_file, sheetname='renters')
renters_df

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Ivy,30000,10000,0,262 That St,Mobile Home,1000,1,1,700,1920,100000,Extensive,Rent,Rent,Single Family Dwelling
1,Edward,100000,50000,10000000,4720 This Rd,Single Family Dwelling,2500,3,2,5000,1920,10000000,Complete,Own,Rent,Single Family Dwelling
2,Oswald,10000,100,0,2301 Other Ave,Multi Family Dwelling,750,0,1,250,1960,10000,Complete,Rent,Rent,Multi Family Dwelling
3,James,50000,1000,550000,74000 Out There Lane,Single Family Dwelling,2000,2,2,2000,2010,800000,Complete,Own,Rent,Single Family Dwelling


In [369]:
# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]
human_cap_data

Type
Contractors             40
Engineers               20
FEMA Processors         40
Inspectors               4
Insurance Adjusters     40
Loan Processors         40
Permit Processors      100
Name: Quantity, dtype: int64

In [370]:
human_cap_data['Contractors'] = 4
human_cap_data['Engineers'] = 4
human_cap_data['FEMA Processors'] = 4
human_cap_data['Inspectors'] = 12
human_cap_data['Insurance Adjusters'] = 4
human_cap_data['Loan Processors'] = 4
human_cap_data['Permit Processors'] = 4

In [371]:
# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]
financial_cap_data

Type
Building Materials     2000000
FEMA Aid              35000000
Name: Quantity, dtype: int64

In [372]:
financial_cap_data['Building Materials'] = 2000000
financial_cap_data['FEMA Aid'] = 2000000

In [373]:
# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
forsale_stock_df = pd.read_excel(scenario_file, sheetname='forsale_stock')
forsale_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 New Ave,Mobile Home,100,1,1,700,1920,99999,None
1,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,None
2,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,None
3,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,None


In [374]:
# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
forrent_stock_df = pd.read_excel(scenario_file, sheetname='forrent_stock')
forrent_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 Old Ave,Mobile Home,100,1,1,700,1920,99999,Complete
1,101 Old Ave,Single Family Dwelling,100000,2,2,5000,1920,9999,Complete
2,102 Old Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Complete
3,103 Old Ave,Single Family Dwelling,2000,3,2,2000,2010,800000,Complete


## Simulation Initiation

__Set simulation environment__

In [375]:
simulation = simpy.Environment()

__Indicate whether want to keep track of the stories of each entity (household) in the simulation.__

In [376]:
write_story = True

__Populate the simulation with human and financial capital data.__

In [377]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

human_capital = capitals.HumanCapital(simulation, human_cap_data)

__Create and populate a FilterStore as a vacant housing stock.__

In [378]:
forsale_stock = capitals.importHousingStock(simulation, forsale_stock_df)

__Create and populate a FilterStore as a vacant rental stock.__

In [379]:
forrent_stock = capitals.importHousingStock(simulation, forrent_stock_df)

__Create an empty FilterStore to use as the occupied housing stock.__

In [380]:
owned_stock = FilterStore(simulation)
rented_stock = FilterStore(simulation)

__Import data on households. Everytime a household's residence is created put it in the occupied housing stock.__

In [381]:
owners = entities.importOwners(simulation, owned_stock, owners_df, write_story)

In [382]:
renters = entities.importRenters(simulation, rented_stock, renters_df, write_story)

In [383]:
landlords = []

for renter in renters:
    landlords.append(entities.Landlord())

In [384]:
entities.assignLandlords(renters, landlords, write_story)

In [385]:
renters[1] == renters[1].landlord.tenant

True

In [386]:
forrent_stock.get_queue

[]

In [387]:
renters[1].residence == renters[1].landlord.residence

True

__Write a master process that combines process and functions from search, rebuild, and request modules.__

In [388]:
def landlord_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
            entity.residence = []
            entity.tenant.prior_residence = entity.tenant.residence
            entity.tenant.residence = []
            return
        
        # If home is completely damaged, evict tenant
        if entity.residence.damage_state == 'Complete':
            entity.residence = []
            entity.tenant.prior_residence = entity.tenant.residence
            entity.tenant.residence = []
            
            if write_story == True:
                entity.tenant.story.append(
                '{0} was permanently evicted because the {1} was demolished. '.format(
                entity.tenant.name, entity.tenant.prior_residence.occupancy.lower())
                )
            return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
            
            vacancy_level = len(forrent_stock.items) - len(forrent_stock.get_queue)
            vacancy_rate = vacancy_level / len(forrent_stock.items)
            rent_delta = 9.08 - 0.21 * vacancy_rate
              
            print(len(forrent_stock.items))
            print(len(forrent_stock.get_queue))
            print(vacancy_level)
            print(vacancy_rate)
            print(rent_delta)
               
   
   

In [389]:
def owner_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, forsale_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
            
            yield simulation.process(request.reoccupy(simulation, entity, write_story))
               

In [390]:
def renter_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(landlord_process(simulation, human_capital, financial_capital, entity.landlord, write_story))

    if entity.residence:
        yield simulation.process(request.reoccupy(simulation, entity, write_story))
    
    if not entity.residence:
        search_patience = 550  # days until give up the search for a new home
        search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, search_patience, forrent_stock, human_capital, write_story))

In [391]:
# Initiate a master process for each owner to be modeled in the simulation
for i in range(len(owners)):
    simulation.process(owner_process(simulation, human_capital, financial_capital, owners[i], write_story))

In [392]:
# Initiate a master process for each renter to be modeled in the simulation
for i in range(len(renters)):
    simulation.process(renter_process(simulation, human_capital, financial_capital, renters[i], write_story))

__Can do some cool stuff with the vacant housing stock.__

In [393]:
# Do inspections on all of the for sale stock
for home in forsale_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability
fix_probability = 1.0
fix_schedule = 100

start_delayed(simulation, rebuild.stock(simulation, forsale_stock, fix_probability), fix_schedule)

<Process(starter) object at 0x1149966a0>

In [394]:
# # Do inspections on all of the rental stock
# for home in forrent_stock.items:
#     simulation.process(request.inspection(simulation, human_capital, home))

# # Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# # with probability = fix_probability
# fix_probability = 0.0
# fix_schedule = 200

# start_delayed(simulation, rebuild.stock(simulation, forrent_stock, fix_probability), fix_schedule)

In [395]:
simulation.run()

1 homes in the vacant building stock were fixed on day 100.
4
3
1
0.25
9.0275


## Outputs

__Owner summary statistics__

In [396]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_homesearch = 0
num_gave_up_home_search = 0

for household in owners:
    if household.money_search_start > 0.0: num_damaged += 1
    if household.home_get > 0.0: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1
    if household.home_search_start > 0.0: num_homesearch += 1
    if household.home_search_stop > 0.0: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} owners suffered damage to their homes.\n'.format(num_damaged, len(owners)),
      '{0} out of {1} owners rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(owners)),
        '{0} out of {1} owners gave up searching for money.\n'.format(num_gave_up_money_search, len(owners)),
      '{0} out of {1} owners searched for a new home.\n'.format(num_homesearch, len(owners)),
        '{0} out of {1} owners bought a new home.\n'.format(num_relocated, len(owners)),
        '{0} out of {1} owners gave up searching for a home.'.format(num_gave_up_home_search, len(owners))
      )

1 out of 4 owners suffered damage to their homes.
 0 out of 4 owners rebuilt or repaired their damaged home.
 0 out of 4 owners gave up searching for money.
 1 out of 4 owners searched for a new home.
 1 out of 4 owners bought a new home.
 0 out of 4 owners gave up searching for a home.


__Owner stories__

In [397]:
owners[0].story

['Alfred owns and lives in a 1 bedroom mobile home at 62 That St worth $100,000. ',
 "Alfred's mobile home was inspected 10 days after the event and suffered $100,000 of damage.",
 'Alfred submitted a request to FEMA 10 days after the event. ',
 'Alfred received $100,000 from FEMA 30 days after the event. ',
 'It took Alfred 20 days to receive enough financial assistance and now has $100,000 to rebuild. ',
 'Alfred started searching for a new mobile home 30 days after the event. ',
 'On day 30, Alfred purchased a mobile home at 100 New Ave with a value of $99,999 and $0 of damage. ']

In [398]:
owners[1].story

['Bruce owns and lives in a 5 bedroom single family dwelling at 720 This Rd worth $10,000,000. ',
 "Bruce's single family dwelling was inspected 10 days after the event and suffered $0 of damage."]

In [399]:
owners[2].story

['Selena owns and lives in a 0 bedroom single family dwelling at 1001 Other Ave worth $10,000. ',
 "Selena's single family dwelling was inspected 10 days after the event and suffered $0 of damage."]

In [400]:
owners[3].story

['Fish owns and lives in a 4 bedroom single family dwelling at 26000 Out There Lane worth $800,000. ',
 "Fish's single family dwelling was inspected 10 days after the event and suffered $0 of damage."]

__Renter summary statistics__

In [401]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_displaced = 0
num_gave_up_home_search = 0

for renter in renters:

    if renter.landlord.money_search_start > 0.0: num_damaged += 1
    if renter.landlord.home_get > 0.0: num_rebuilt += 1
    if renter.landlord.gave_up_money_search: num_gave_up_money_search += 1
    if not renter.residence: num_displaced += 1
        
print('{0} out of {1} renters\' homes suffered damage.\n'.format(num_damaged, len(renters)),
      '{0} out of {1} renters\' damaged home was rebuilt or repaired.\n'.format(num_rebuilt, len(renters)),
      '{0} out of {1} renters\' were displaced.\n'.format(num_displaced, len(renters)),
      )

4 out of 4 renters' homes suffered damage.
 1 out of 4 renters' damaged home was rebuilt or repaired.
 3 out of 4 renters' were displaced.



__Renters Stories__

In [402]:
renters[0].story + renters[0].landlord.story

['Ivy rents and lives in a 1 bedroom mobile home at 262 That St worth $100,000. ',
 "Ivy's residence is owned by Roger. ",
 'Ivy reoccupied the mobile home 137 days after the event. ',
 "Roger's mobile home was inspected 10 days after the event and suffered $37,500 of damage.",
 'Roger submitted an insurance claim 10 days after the event. ',
 'Roger received a $22,500 insurance payout 25 days after the event. ',
 'Roger submitted a request to FEMA 25 days after the event. ',
 'Roger received $15,000 from FEMA 45 days after the event. ',
 'It took Roger 35 days to receive enough financial assistance and now has $37,500 to rebuild. ',
 'Roger received an engineering assessment 70 days after the event. ',
 'Roger received permit approval 105 days after the event. ',
 "Roger's mobile home was repaired 135 days after the event, taking 30 days to repair. "]

In [403]:
renters[1].story + renters[1].landlord.story

['Edward rents and lives in a 3 bedroom single family dwelling at 4720 This Rd worth $10,000,000. ',
 "Edward's residence is owned by William. ",
 'Edward was permanently evicted because the single family dwelling was demolished. ',
 'Edward started searching for a new single family dwelling 45 days after the event. ',
 'On day 595, after a 550 day search, Edward gave up looking for a new home in the local area. ',
 "William's single family dwelling was inspected 10 days after the event and suffered $10,000,000 of damage.",
 'William submitted an insurance claim 10 days after the event. ',
 'William received a $8,500,000 insurance payout 25 days after the event. ',
 'William submitted a request to FEMA 25 days after the event. ',
 'William received $1,500,000 from FEMA 45 days after the event. ',
 'It took William 35 days to receive enough financial assistance and now has $10,000,000 to rebuild. ']

In [404]:
renters[2].story + renters[2].landlord.story

['Oswald rents and lives in a 0 bedroom multi family dwelling at 2301 Other Ave worth $10,000. ',
 "Oswald's residence is owned by James. ",
 'Oswald was permanently evicted because the multi family dwelling was demolished. ',
 'Oswald started searching for a new multi family dwelling 45 days after the event. ',
 'On day 595, after a 550 day search, Oswald gave up looking for a new home in the local area. ',
 "James's multi family dwelling was inspected 10 days after the event and suffered $10,000 of damage.",
 'James submitted an insurance claim 10 days after the event. ',
 'James received a $8,500 insurance payout 25 days after the event. ',
 'James submitted a request to FEMA 25 days after the event. ',
 'James received $1,500 from FEMA 45 days after the event. ',
 'It took James 35 days to receive enough financial assistance and now has $10,000 to rebuild. ']

In [405]:
renters[3].story + renters[3].landlord.story

['James rents and lives in a 2 bedroom single family dwelling at 74000 Out There Lane worth $800,000. ',
 "James's residence is owned by Alfredo. ",
 'James was permanently evicted because the single family dwelling was demolished. ',
 'James started searching for a new single family dwelling 50 days after the event. ',
 'On day 600, after a 550 day search, James gave up looking for a new home in the local area. ',
 "Alfredo's single family dwelling was inspected 10 days after the event and suffered $800,000 of damage.",
 'Alfredo submitted an insurance claim 10 days after the event. ',
 'Alfredo received a $680,000 insurance payout 25 days after the event. ',
 'Alfredo submitted a request to FEMA 25 days after the event. ',
 'Alfredo received $120,000 from FEMA 50 days after the event. ',
 'It took Alfredo 40 days to receive enough financial assistance and now has $800,000 to rebuild. ']

In [406]:
renters[1].residence.cost

AttributeError: 'list' object has no attribute 'cost'

In [ ]:
if not renters[1].residence: print('Im homeless.')

In [ ]:
isinstance(renters[1], entities.Renter)

In [ ]:
renters[2].residence